#### 이진분류 파이토치

In [3]:
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [ ]:
### 이진 교차 엔트로피

device = torch.device("mps")
criterion = nn.BCELoss().to(device)

In [6]:
# 전체 코드

import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self,x):
        x = self.layer(x)
        return x
    
dataset = CustomDataset("./dataset/binary.csv")
dataset_size = len(dataset)

train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

# random_split 함수를 사용하여 주어진 데이터세를 훈련 검증 테스트 세트로 무작위 분할
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4)) # torch.manual_seed 고정된 난수 발생기 시드 (재현성 보장)

# 배치 단위로 데이터 로드
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size = 4, shuffle=True, drop_last=True)

device = torch.device('mps')
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.00001)

for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost: .3f}')

# 검증
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)

        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("-------------------------------------")

Epoch : 1000, Model : [Parameter containing:
tensor([[-0.4287, -0.3430,  0.1208]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2057], device='mps:0', requires_grad=True)], Cost :  23.327
Epoch : 2000, Model : [Parameter containing:
tensor([[ 0.0020, -0.0011,  0.0036]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2007], device='mps:0', requires_grad=True)], Cost :  0.691
Epoch : 3000, Model : [Parameter containing:
tensor([[ 0.0026, -0.0009,  0.0029]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1873], device='mps:0', requires_grad=True)], Cost :  0.687
Epoch : 4000, Model : [Parameter containing:
tensor([[ 0.0027, -0.0008,  0.0030]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.1740], device='mps:0', requires_grad=True)], Cost :  0.684
Epoch : 5000, Model : [Parameter containing:
tensor([[ 0.0027, -0.0007,  0.0031]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.160

#### 비선형 활성화 함수